In [1]:
import requests
import os, json
from IPython.display import JSON
import base64
from datetime import datetime

In [2]:
api_endpoint = "https://staging.harritydev.com/api/uspto/applications/"
#analyze_oa_url = "https://staging.harritydev.com/api/nlp/api/v1/parser/analyze/office-action"
analyze_oa_url = "http://localhost:5000/api/v1/parser/analyze/office-action"
root_folder = "data/oa_samples"

staging_bearer_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6ImFiYWxhYmFuQGhhcnJpdHlkZXYub25taWNyb3NvZnQuY29tIiwiZmlyc3ROYW1lIjoiQW5kcmlpIiwibGFzdE5hbWUiOiJCYWxhYmFuIE9mZmljZSAzNjUiLCJwZXJtaXNzaW9ucyI6WyJwb3J0Zm9saW8iLCJ1c2VycyIsIm9yZ2FuaXphdGlvbnMiLCJzcGVjaWZpY2F0aW9ucyIsImZvcm1zIiwidGVtcGxhdGVzIiwiYXBwbGljYXRpb25zIiwidXNwdG8iLCJtZXNzYWdlcyIsIndvcmtmbG93cyIsImFkbWluaXN0cmF0aW9uIiwiZG9ja2V0aW5nX3Rhc2tzIiwiYWRtaW5pc3RyYXRpb25fdGFza3MiLCJib3Rfc2NyaXB0cyJdLCJ0ZWFtcyI6WyJEcmFmdGluZyBUZWFtIl0sIm1zU2VjdXJpdHlHcm91cHMiOlt7ImlkIjoiODY3NmJjY2MtZjMzYi00YTUyLTg2NGYtZjA0ZDM1NjI5Zjc1Iiwic2VjdXJpdHlJZGVudGlmaWVyIjoiUy0xLTEyLTEtMjI1NTkyODUyNC0xMjQ2OTUwMjAzLTEzMDc1OTQ2MzAtMTk3MzM3OTYzNyIsImRpc3BsYXlOYW1lIjoiU29tZU90aGVyR3JvdXAiLCJvblByZW1pc2VzTmFtZSI6bnVsbH0seyJpZCI6ImNiYTA4ODdkLWQzYTMtNGY4ZS04YWI3LWM1NWQ3YWRkMmZiZSIsInNlY3VyaXR5SWRlbnRpZmllciI6IlMtMS0xMi0xLTM0MTYyOTU1NDktMTMzNDc2MDM1NS0xNTczMjM4NjY2LTMxOTA4MDc5MzAiLCJkaXNwbGF5TmFtZSI6IkFub3RoZXJHcm91cCIsIm9uUHJlbWlzZXNOYW1lIjpudWxsfV0sImlhdCI6MTY4NDkzODk1NiwiZXhwIjoxNjg3NTMwOTU2LCJzdWIiOiI2MmQ5ODA0Y2U5MjMyN2FiZjljN2QxMTYifQ.qI5Pfh_rwwFJszRBAahYucbN3C3g-1ABQBe27gzyWPo"
dev_bearer_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6ImFiYWxhYmFuQGhhcnJpdHlkZXYub25taWNyb3NvZnQuY29tIiwiZmlyc3ROYW1lIjoiQW5kcmlpIiwibGFzdE5hbWUiOiJCYWxhYmFuIE9mZmljZSAzNjUiLCJwZXJtaXNzaW9ucyI6WyJwb3J0Zm9saW8iLCJ1c2VycyIsIm9yZ2FuaXphdGlvbnMiLCJzcGVjaWZpY2F0aW9ucyIsImZvcm1zIiwidGVtcGxhdGVzIiwiYXBwbGljYXRpb25zIiwidXNwdG8iLCJtZXNzYWdlcyIsIndvcmtmbG93cyIsImFkbWluaXN0cmF0aW9uIiwiZG9ja2V0aW5nX3Rhc2tzIiwiYWRtaW5pc3RyYXRpb25fdGFza3MiLCJib3Rfc2NyaXB0cyJdLCJ0ZWFtcyI6WyJEcmFmdGluZyBUZWFtIl0sIm1zU2VjdXJpdHlHcm91cHMiOlt7ImlkIjoiODY3NmJjY2MtZjMzYi00YTUyLTg2NGYtZjA0ZDM1NjI5Zjc1Iiwic2VjdXJpdHlJZGVudGlmaWVyIjoiUy0xLTEyLTEtMjI1NTkyODUyNC0xMjQ2OTUwMjAzLTEzMDc1OTQ2MzAtMTk3MzM3OTYzNyIsImRpc3BsYXlOYW1lIjoiU29tZU90aGVyR3JvdXAiLCJvblByZW1pc2VzTmFtZSI6bnVsbH0seyJpZCI6ImNiYTA4ODdkLWQzYTMtNGY4ZS04YWI3LWM1NWQ3YWRkMmZiZSIsInNlY3VyaXR5SWRlbnRpZmllciI6IlMtMS0xMi0xLTM0MTYyOTU1NDktMTMzNDc2MDM1NS0xNTczMjM4NjY2LTMxOTA4MDc5MzAiLCJkaXNwbGF5TmFtZSI6IkFub3RoZXJHcm91cCIsIm9uUHJlbWlzZXNOYW1lIjpudWxsfV0sImlhdCI6MTY4NDkzODk1NiwiZXhwIjoxNjg3NTMwOTU2LCJzdWIiOiI2MmQ5ODA0Y2U5MjMyN2FiZjljN2QxMTYifQ.qI5Pfh_rwwFJszRBAahYucbN3C3g-1ABQBe27gzyWPo"

# Create the headers dictionary with the authorization header
headers = {
    "Authorization": f"Bearer {staging_bearer_token}"
}

os.makedirs(root_folder, exist_ok=True)

In [28]:
application = "16949396"
try:
    response = requests.get(api_endpoint+application, headers=headers)
    response.raise_for_status()
    file_urls = response.json()

except requests.exceptions.HTTPError as e:
    print(f"An HTTP error occurred: {e}")

In [29]:
docList = file_urls["documents"]

In [30]:
JSON(docList)

<IPython.core.display.JSON object>

In [31]:
OAs = [doc for doc in file_urls["documents"] 
       if doc["code"]=="CTNF" or doc["code"]=="CTFR"]

In [32]:
JSON(OAs)

<IPython.core.display.JSON object>

In [33]:
CLMs = [doc for doc in docList if doc["code"]=="CLM"]


In [34]:
JSON(CLMs)

<IPython.core.display.JSON object>

In [35]:
def getPreviousClaims(oa, claims):
    threshold_date = datetime.strptime(oa["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
    sorted_claims = sorted(claims, key=lambda x: datetime.strptime(x["date"],
                            "%Y-%m-%dT%H:%M:%S.%fZ"), reverse=True)
    for clm in sorted_claims:
        clm_date = datetime.strptime(clm["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
        if clm_date < threshold_date:
            return clm

In [36]:
def download_file(name):
    try:
        response = requests.get(api_endpoint+application+"/documents/" + 
                                name, headers=headers)
        response.raise_for_status()

    except requests.exceptions.HTTPError as e:
        print(f"An HTTP error occurred: {e}")
    
    return response.content

In [37]:
def save_binary(folder, name, content):
    with open(folder + "/" + name, "wb") as f:
        f.write(content)

In [38]:
def save_json(folder, name, content):
    type(content)
    if isinstance(content, bytes):
        content = base64.b64encode(content).decode('utf-8')
    with open(folder + "/" + name + ".json", "w") as f:
        json.dump(content, f, indent=4)

In [39]:
def save_pdf(folder, name, content):
    with open(folder + "/" + name + ".pdf", "wb") as f:
        f.write(content)

In [40]:
def analyze_oa(oa_artifact, clm_artifact):
    payload = {"artifacts": [oa_artifact, clm_artifact]}
    save_json(output_folder, "request_body", payload)
    
    headers = {
        "Authorization": f"Bearer {staging_bearer_token}"
    }
    
    try:
        response = requests.post(analyze_oa_url, json=payload, headers=headers)
        response.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        print(f"An HTTP error occurred: {e}")
    save_json(output_folder, "current_context", response.json())    
    return response

In [41]:
for OA in OAs[:1]:
    oa_date_str = OA["date"].split('T')[0]
    output_folder = (root_folder + "/" + application + 
                    "-" + oa_date_str + "-" + OA["code"])
    os.makedirs(output_folder, exist_ok=True)
    
    oa = download_file(OA["id"] + ".xml")
    save_binary(output_folder, oa_date_str + 
         "-" + OA["code"] + ".xml", oa)

    oapdf = download_file(OA["id"] + ".pdf")
    save_binary(output_folder, oa_date_str + 
         "-" + OA["code"] + ".pdf", oapdf)
 
    CLM = getPreviousClaims(OA, CLMs)
    
    clm = download_file(CLM["id"] + ".xml")
    save_binary(output_folder, CLM["date"].split('T')[0] + 
             "-" + CLM["code"] + ".xml", clm)
    
    clmpdf = download_file(CLM["id"] + ".pdf")
    save_binary(output_folder, CLM["date"].split('T')[0] + 
             "-" + CLM["code"] + ".pdf", clmpdf)

    oa64 = base64.b64encode(oa).decode('utf-8')
    clm64 = base64.b64encode(clm).decode('utf-8')

    context = analyze_oa({"type": "oaxml", "base64": oa64}, 
               {"type": "clmxml", "base64": clm64})


In [42]:
JSON(context.json())

<IPython.core.display.JSON object>

In [25]:
def generate(template, context):
    payload = {
        "templates": [
            {
                "name": "test",
                "body": template,
                "claims": [],
                "params": []
            }
        ],
        "context": context
    }
    
    headers = {
        "Authorization": f"Bearer {dev_bearer_token}"
    }
    
    try:
        #response = requests.post("https://staging.harritydev.com/api/nlp/api/v1/generator/generate", json=payload, headers=headers)
        response = requests.post("http://localhost:5000/api/v1/generator/generate", json=payload, headers=headers)
        response.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        print(f"An HTTP error occurred: {e}")
    save_json(output_folder, "current_output", response.json())    
    return response


In [19]:
generate("hello {{examiner.firstName}}, can I call you at {{examiner.phone}}?", context.json()).json()["paragraphs"][0]["runs"][0]["text"]

'hello SETH, can I call you at (571)272-5105?'

In [ ]:
generate("Claims {{listToRanges indexOfClaims}} have been filed.", context.json()).json()["paragraphs"][0]["runs"][0]["text"]

In [ ]:
generate("hello {{#each indexOfClaims}} {{hasDispositionNotAppliedToIndParent this}} {{/each}}", context.json()).json()["paragraphs"][0]["runs"][0]["text"]

In [ ]:
generate("hello {{listToRanges (filter indexOfClaims 'hasDispositionNotAppliedToIndParent')}}", context.json()).json()["paragraphs"][0]["runs"][0]["text"]

In [ ]:
generate("""{{#each (groupByIndependentClaim (filter indexOfClaims 'hasDispositionNotAppliedToIndParent'))}}
              Claim {{independentClaimNumber}} has dependent claims {{listToRanges dependentClaims}}. 
              {{/each}}"""
              , context.json()).json()["paragraphs"][0]["runs"][0]["text"]